# Crew to tailor job application

In [2]:
from crewai import Agent, Task, LLM, Crew
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
serper_api_key = os.getenv("SERPER_API_KEY")



In [9]:
myllm = LLM(
    model = "gemini/gemini-2.0-flash",
    api_key = os.getenv("GOOGLE_API_KEY")
)

In [21]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(
    config=dict(
        llm=dict(
            provider="google", # Options include google, openai, anthropic, llama2, etc.
            config=dict(
                model="gemini-2.0-flash",
                api_key = os.getenv("GOOGLE_API_KEY")
                # Optional parameters can be included here.
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # Optional title for the embeddings can be added here.
                # title="Embeddings",
                
            ),
        ),
    )
    ,mdx='/home/avinash/Inexture/AI/Gen AI/Practice/fake_resume.md'
)


In [22]:
researcher = Agent(
    role = "Tech Job Reasearcher",
    goal = "Make sure to do amazing analysis on job posting to"
    "help job applicants",
    tools = [scrape_tool, search_tool],
    verbose = True,
    backstory = (
        "As a Job Reasearcher, your prowess in navigating"
        "and extracting critical information"
        "from job posting is unmatched."
        "Your skills help pinpoint the necessary"
        "qualification and skills sought by employers,"
        "forming the foundation for effective application tailoring."
    )

)

In [25]:
profiler = Agent(
    role = "Personal Profiler for Engineers",
    goal = "Do incredible research on job applicants"
    "to help them standout in the job market",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose = True,
    backstory = (
        "Equipped with analytical prowess,"
        "you dissect and synthesize information"
        "from diverse sources to craft comprehensive"
        "personal and professional profiles,"
        "laying the ground work for personalized resume enhancements."
    )
)

In [27]:
resume_strategist = Agent(
    role = "Resume Strategist for Engineers",
    goal = "Find all the best ways to make"
    "a resume stand out in the job market.",
    tools = [scrape_tool, search_tool, read_resume, semantic_search_resume],
    verbose = True,
    backstory = (
        "With a strategic mind and an eye for detail, "
        "you excel at refining resumes to highlight"
        "the most relevant skills and experiences,"
        "ensuring they resonate perfectly with"
        "the job's requiremnts."
    )
    
)

#  Creaing TAsk

In [28]:
research_task = Task(
    description = (
        "Analyze the job posting URL provided"
        "{job_posting_url} to extract key skills, experiences,"
        "and qualifications required."
        "Use the tools to gather content and "
        "identify and categorize the requirements."
    ),

    expected_output = (
        "A structured list of job requirements,"
        "including all necessary skills,"
        "qualification, and experiences."
    ),
    agent = researcher,
    async_execution = True
)

In [ ]:
profile_task = Task(
    description = (
        "Compile a detailed personal and professional profile"
        "using the GitHub ({github_url}) URLs,"
        "and personal write-up ({personal_writeup})."
        "Utilize tools to extract and synthesize information"
        "from these sources."
    ),
    expected_output = (
        "A comprehensive profile document that",
        "includes skills, project experinces,"
        "contributions, interests, and communication style."
    ),
    agent = profiler,
    async_execution = True
)